In [3]:
import os
import sys
import pandas as pd
import numpy as np
from numpy.polynomial import Polynomial as P
filepath = os.path.dirname(os.path.abspath("")) + "\\ToPython"
import matplotlib.pyplot as plt
import sympy
%matplotlib inline
# ERRO norm(VETOR_ORIGINAL - VETOR_GERADO_PELO_POLINOMIO).^2 / norm(VETOR_ORIGINAL).^2
#https://www.youtube.com/watch?v=_pJX2SFoFtY
# https://www.studytonight.com/post/what-is-mean-squared-error-mean-absolute-error-root-mean-squared-error-and-r-squared
days = 23

In [2]:
# gerar funçoes polinomiais e salvar em um csv
tipos= ['bytes','pacotes']
polys = []
xm = list(range(1440))
for tipo in tipos:
    filename = f'dados_{tipo}.csv'
    fullpath = filepath +'/sharedData/'+ filename

    df = pd.read_csv(fullpath)
    lista = []
    for day in list(range(days)):
        day+=1
        print(day-1)
        day = df.iloc[day-1][1:1441]
        ym = np.array(day)
        p = P.fit(xm, ym, 12)
        lista.append(p)
    poly = pd.DataFrame(lista)
    poly.to_csv(f"poly_{tipo}.csv")
    polys.append(poly)
    list_predict =[]
    # gerar dados previstos
    for day in lista:
        predict = []
        for minute in list(range(1440)):
            predict.append(day(minute))
        list_predict.append(predict)
        df = pd.DataFrame(list_predict)
        df.to_csv(f"predict_{tipo}.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [3]:
# create csvs according to interval 1,5,10,30 of real data
list_interval = [1,5,10,30]
tipos= ['bytes','pacotes']
dayp = []
for tipo in tipos:
    filename = f'{filepath}\sharedData\dados_{tipo}.csv'
    df = pd.read_csv(filename)
    for interval in list_interval:
        
        dfaux = pd.DataFrame()
        for day in list(range(len(df))):
            dayr = df.iloc[day][1:]
            size = int(len(dayr)/interval)
            for index in list(range(size)):
                ini = int(index*interval)
                fim = int(index*interval+interval)
                soma = sum(dayr[ini:fim])
                media = soma/interval
                dayp.append(media)
#             print(len(dayp))
            dfaux[day] = dayp
            dayp = []
        filename = f"dados_{str(interval)}_{tipo}_real.csv"
        dfaux.to_csv(filename, header = False)

In [5]:
# create csvs/graphs according to interval 1,5,10,30 of predict data
def plot_graph(ym,p,interval,tipo,titulo):
    xm = list(range(int(1440/interval)))
    xm1 = [x*60/interval for x in list(range(25))]
    ticks = [x for x in list(range(25))]
    plt.xticks(xm1, ticks)
    plt.plot(xm,ym,'b',label = 'Dados reais')
    plt.plot(*p.linspace(int(1440/interval)),'r',label = 'Previsão(Polinomial)')
    plt.xlabel("Horas")
    plt.ylabel(f"{tipo}/s")
    plt.title(titulo)
    plt.legend()
    return plt
    plt.show()
    
def savel_plots(day,erro,ym1,p,interval,tipo):
    week_days = {
        1:'Sexta',
        2:'Sabado',
        3:'Domingo',
        4:'Segunda',
        5:'Terça',
        6:'Quarta',
        7:'Quinta',
    }
    erro_rounded = round(erro,4)
    titulo = f'{week_days[day]}\nIntervalo:{interval}min\nErro Quadrático:{erro_rounded}'
    plt = plot_graph(ym1,p,interval,tipo,titulo)
    filename = f"{tipo}Intervalo{interval}.png"
    filepath = f"dados/{day}/aux{filename}"
    plt.savefig(filepath,bbox_inches='tight',dpi=300)
    plt.close()
###############################################################################################
list_interval = [1,5,10,30]

tipos = ["bytes","pacotes"]
erro_all = []
for tipo in tipos:
    erro_intervals = []
    for interval in list_interval:
        
        xm = list(range(int(1440/interval)))
        filename = f'dados_{interval}_{tipo}_real.csv'
        df = pd.read_csv(filename,header=None)
        df = df.drop(columns =[0])
        dfaux = pd.DataFrame()
        erro_week = []
        for index in list(range(7)):
            
            print(f"Tipo {tipo}")
            print(f"Intervalo {interval}")
            print(f"DIA {index}")
            index+=1
            dayt1 = df[index]
            dayt2 = df[index+7]
            dayv = df[index+14]
            
            ym1 = np.array(dayt1)
            ym2 = np.array(dayt2)
            p1 = P.fit(xm, ym1, 12)
            p2 = P.fit(xm, ym2, 12)
            
            p = (p1+p2)/2
            
            dayr = []
            
            for each in list(range(int(1440/interval))):
                dayr.append(p(each))
            erro = np.linalg.norm(dayv - dayr)**2/np.linalg.norm(dayv)**2
            erro_week.append(erro)
            day = index
            savel_plots(day,erro,dayv,p,interval,tipo)
        erro_intervals.append(erro_week)
    pd.DataFrame(erro_intervals).to_csv(f"dados/erro_poly_{tipo}.csv")
    erro_all.append(erro_intervals)

Tipo bytes
Intervalo 1
DIA 0
Tipo bytes
Intervalo 1
DIA 1
Tipo bytes
Intervalo 1
DIA 2
Tipo bytes
Intervalo 1
DIA 3
Tipo bytes
Intervalo 1
DIA 4
Tipo bytes
Intervalo 1
DIA 5
Tipo bytes
Intervalo 1
DIA 6
Tipo bytes
Intervalo 5
DIA 0
Tipo bytes
Intervalo 5
DIA 1
Tipo bytes
Intervalo 5
DIA 2
Tipo bytes
Intervalo 5
DIA 3
Tipo bytes
Intervalo 5
DIA 4
Tipo bytes
Intervalo 5
DIA 5
Tipo bytes
Intervalo 5
DIA 6
Tipo bytes
Intervalo 10
DIA 0
Tipo bytes
Intervalo 10
DIA 1
Tipo bytes
Intervalo 10
DIA 2
Tipo bytes
Intervalo 10
DIA 3
Tipo bytes
Intervalo 10
DIA 4
Tipo bytes
Intervalo 10
DIA 5
Tipo bytes
Intervalo 10
DIA 6
Tipo bytes
Intervalo 30
DIA 0
Tipo bytes
Intervalo 30
DIA 1
Tipo bytes
Intervalo 30
DIA 2
Tipo bytes
Intervalo 30
DIA 3
Tipo bytes
Intervalo 30
DIA 4
Tipo bytes
Intervalo 30
DIA 5
Tipo bytes
Intervalo 30
DIA 6
Tipo pacotes
Intervalo 1
DIA 0
Tipo pacotes
Intervalo 1
DIA 1
Tipo pacotes
Intervalo 1
DIA 2
Tipo pacotes
Intervalo 1
DIA 3
Tipo pacotes
Intervalo 1
DIA 4
Tipo pacotes
Interv

In [12]:
dayv

0       25442577.2
1       17134624.4
2       17794498.0
3       14704596.0
4       21140480.0
           ...    
1435     6216284.1
1436     9665135.9
1437    13459772.0
1438     9103414.1
1439    15212016.1
Name: 15, Length: 1440, dtype: float64

In [5]:
# Cria os arquivos reais de csv de entropia
list_interval = [1,5,10,30]

for interval in list_interval:
    ip_origem= []
    ip_destino=[]
    porta_origem =[]
    porta_destino =[]
    pacotes_ps =[]
    bytes_ps=[]
    interval*=60
    interval = str(interval)
    for day in list(range(days)):
        day+=1
        filepath2 = f'{filepath}\\Backup\\Dados\\DadosATUALIZADOS\\entropy\\{interval}\\{day}.csv'
        df = pd.read_csv(filepath2)        
        ip_origem.append(df['ip_origem'])
        ip_destino.append(df['ip_destino'])
        porta_origem.append(df['porta_origem'])
        porta_destino.append(df['porta_destino'])
        pacotes_ps.append(df['pacotes_ps'])
        bytes_ps.append(df['bytes_ps'])
    interval = str(int(int(interval)/60))
    pd.DataFrame(ip_origem).to_csv(f"dadosEntropy/ip_origem_{interval}.csv", header = False)
    pd.DataFrame(ip_destino).to_csv(f"dadosEntropy/ip_destino_{interval}.csv", header = False)
    pd.DataFrame(porta_origem).to_csv(f"dadosEntropy/porta_origem_{interval}.csv", header = False)
    pd.DataFrame(porta_destino).to_csv(f"dadosEntropy/porta_destino_{interval}.csv", header = False)
    pd.DataFrame(pacotes_ps).to_csv(f"dadosEntropy/pacotes_ps_{interval}.csv", header = False)
    pd.DataFrame(bytes_ps).to_csv(f"dadosEntropy/bytes_ps_{interval}.csv", header = False)
        
    

In [6]:
# Entropia
# create csvs according to interval 1,5,10,30 of predict data
def plot_graph(ym,p,interval,tipo,titulo):
    xm = list(range(int(1440/interval)))
    xm1 = [x*60/interval for x in list(range(25))]
    ticks = [x for x in list(range(25))]
    plt.xticks(xm1, ticks)
    plt.plot(xm,ym,'b',label = 'Dados reais')
    plt.plot(*p.linspace(int(1440/interval)),'r',label = 'Previsão(Polinomial)')
    plt.xlabel("Horas")
    plt.ylabel(f"{tipo}/s")
    plt.title(titulo)
    plt.legend()
    return plt
    plt.show()
    
def savel_plots(day,erro,ym1,p,interval,tipo):
    week_days = {
        1:'Sexta',
        2:'Sabado',
        3:'Domingo',
        4:'Segunda',
        5:'Terça',
        6:'Quarta',
        7:'Quinta',
    }
    erro_rounded = round(erro,4)
    titulo = f'{week_days[day]} Intervalo:{interval}min Erro:{erro_rounded}'
    plt = plot_graph(ym1,p,interval,tipo,titulo)
    filename = f"{tipo}Intervalo{interval}.png"
    filepath = f"dadosEntropy/{day}/{filename}"
    plt.savefig(filepath,bbox_inches='tight',dpi=300)
    plt.close()
###############################################################################################
list_interval = [1,5,10,30]

dayp = []
filepath2 = f'{filepath}\\Backup\\Dados\\DadosATUALIZADOS\\entropy\\60\\1.csv'
df = pd.read_csv(filepath2)  
tipos = list(df[['ip_origem','porta_origem','ip_destino','porta_destino','pacotes_ps','bytes_ps']].columns)
erro_all = []
for tipo in tipos:
    erro_intervals = []
    for interval in list_interval:
        xm = list(range(int(1440/interval)))
        filename = f'dadosEntropy/{tipo}_{interval}.csv'
        df = pd.read_csv(filename,header=None)
        df = df.drop(columns =[0])
        dfaux = pd.DataFrame()
        erro_week = []
        for index in list(range(7)):
            
            print(f"Tipo {tipo}")
            print(f"Intervalo {interval}")
            print(f"DIA {index}")
            index+=1
            dayt1 = df.iloc[index]
            dayt2 = df.iloc[index+7]
            dayv = df.iloc[index+14]
            
            ym1 = np.array(dayt1)
            ym2 = np.array(dayt2)
            
            p1 = P.fit(xm, ym1, 12)
            p2 = P.fit(xm, ym2, 12)
            p = (p1+p2)/2
            
            dayr = []
            
            for each in list(range(int(1440/interval))):
                dayr.append(p(each))
            
            erro = np.linalg.norm(dayv - dayr)**2/np.linalg.norm(dayv)**2
            erro_week.append(erro)
            day = index
            savel_plots(day,erro,dayv,p,interval,tipo)
            
        erro_intervals.append(erro_week)
    pd.DataFrame(erro_intervals).to_csv(f"dadosEntropy/erroPoly/{tipo}.csv")
    erro_all.append(erro_intervals)

Tipo ip_origem
Intervalo 1
DIA 0
Tipo ip_origem
Intervalo 1
DIA 1
Tipo ip_origem
Intervalo 1
DIA 2
Tipo ip_origem
Intervalo 1
DIA 3
Tipo ip_origem
Intervalo 1
DIA 4
Tipo ip_origem
Intervalo 1
DIA 5
Tipo ip_origem
Intervalo 1
DIA 6
Tipo ip_origem
Intervalo 5
DIA 0
Tipo ip_origem
Intervalo 5
DIA 1
Tipo ip_origem
Intervalo 5
DIA 2
Tipo ip_origem
Intervalo 5
DIA 3
Tipo ip_origem
Intervalo 5
DIA 4
Tipo ip_origem
Intervalo 5
DIA 5
Tipo ip_origem
Intervalo 5
DIA 6
Tipo ip_origem
Intervalo 10
DIA 0
Tipo ip_origem
Intervalo 10
DIA 1
Tipo ip_origem
Intervalo 10
DIA 2
Tipo ip_origem
Intervalo 10
DIA 3
Tipo ip_origem
Intervalo 10
DIA 4
Tipo ip_origem
Intervalo 10
DIA 5
Tipo ip_origem
Intervalo 10
DIA 6
Tipo ip_origem
Intervalo 30
DIA 0
Tipo ip_origem
Intervalo 30
DIA 1
Tipo ip_origem
Intervalo 30
DIA 2
Tipo ip_origem
Intervalo 30
DIA 3
Tipo ip_origem
Intervalo 30
DIA 4
Tipo ip_origem
Intervalo 30
DIA 5
Tipo ip_origem
Intervalo 30
DIA 6
Tipo porta_origem
Intervalo 1
DIA 0
Tipo porta_origem
Interval

In [7]:
df.iloc[index]

1     254391.570444
2     257052.827722
3     230174.725833
4     141389.581889
5      94165.781056
6      92732.190389
7      82410.283500
8      76755.692389
9      78580.666444
10     74389.688333
11     55342.920833
12     60119.963167
13     30952.759278
14     31679.363222
15      4824.232222
16      8602.472222
17     10413.766389
18     14516.331278
19     10556.029333
20     16695.069667
21     10522.562722
22      6551.375944
23      4322.362222
24     10580.027333
25     20157.103167
26     16382.487833
27      8666.168556
28      4616.846167
29     13689.219833
30     12283.238611
31     21001.872944
32     54641.584778
33     57356.980389
34     27689.168944
35     76938.455611
36    132936.691722
37    242913.015389
38    325973.697444
39    298419.764167
40    174281.536833
41     98299.750444
42    136914.538667
43    193860.088333
44    350344.331056
45    322674.864111
46    283001.960833
47    301572.606000
48    179967.159611
Name: 7, dtype: float64